#### Generating List of sentences

In [1]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of sentences.
raw_data = corpus.findall('.//sentence')
for sent in raw_data:
    sentences.append(sent.find('text').text)

print ('Generated list of sentences..')

Processing text dataset
Generated list of sentences..


#### Indexing Aspects

In [2]:
import nltk
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

train_out= np.zeros(shape=(3044,69))
i=0
for output in raw_data:
    s = text_to_word_sequence(output.find('text').text, lower=False)
    indices = np.zeros(69)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    indices[s.index(aspect_term.attrib['term'])] = 1
#                     print (indices)
                except:
                    continue
    train_out[i] = indices
    i=i+1

print ("Shape of output tensor:", train_out.shape)

Using Theano backend.
Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


('Shape of output tensor:', (3044, 69))


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


#### Defining input data for CRF Training

In [7]:
train_input = []
i = 0
for sent in sentences:
    tuple_sent = []
    word_token = text_to_word_sequence(sent, lower=False)
    j = 0
    for word in word_token:
        input_tuple = ()
        input_tuple = input_tuple + (word,)
        if train_out[i][j] == 0:
            input_tuple = input_tuple + ('NA',)
        else:
            input_tuple = input_tuple + ('A',)
        tuple_sent.append(input_tuple)
        j=j+1
    train_input.append(tuple_sent)
    i=i+1

In [9]:
train_data = train_input[:2739]
test_data = sentences[2739:]

In [11]:
print train_data[:5]

[[('But', 'NA'), ('the', 'NA'), ('staff', 'A'), ('was', 'NA'), ('so', 'NA'), ('horrible', 'NA'), ('to', 'NA'), ('us', 'NA')], [('To', 'NA'), ('be', 'NA'), ('completely', 'NA'), ('fair', 'NA'), ('the', 'NA'), ('only', 'NA'), ('redeeming', 'NA'), ('factor', 'NA'), ('was', 'NA'), ('the', 'NA'), ('food', 'A'), ('which', 'NA'), ('was', 'NA'), ('above', 'NA'), ('average', 'NA'), ('but', 'NA'), ("couldn't", 'NA'), ('make', 'NA'), ('up', 'NA'), ('for', 'NA'), ('all', 'NA'), ('the', 'NA'), ('other', 'NA'), ('deficiencies', 'NA'), ('of', 'NA'), ('Teodora', 'NA')], [('The', 'NA'), ('food', 'A'), ('is', 'NA'), ('uniformly', 'NA'), ('exceptional', 'NA'), ('with', 'NA'), ('a', 'NA'), ('very', 'NA'), ('capable', 'NA'), ('kitchen', 'A'), ('which', 'NA'), ('will', 'NA'), ('proudly', 'NA'), ('whip', 'NA'), ('up', 'NA'), ('whatever', 'NA'), ('you', 'NA'), ('feel', 'NA'), ('like', 'NA'), ('eating', 'NA'), ('whether', 'NA'), ("it's", 'NA'), ('on', 'NA'), ('the', 'NA'), ('menu', 'A'), ('or', 'NA'), ('not', 

In [ ]:
from nltk.tag import CRFTagger
ct = CRFTagger()

ct.train(train_data,'model.crf.tagger')
ct.tag_sent(test_data)
true_test = train_input[2739:]
ct.evaluate(true_test)